In [0]:
import numpy as np
import heapq as hq
from itertools import count

Every Node object represent a node on the search tree.

It stores
1.   The state: position of all the tiles
2.   A reference to the parent node
3.   g: cost of reaching the node (considering cost of each edge in the tree as 1)
4.   a: represents the action applied on parent node to reach this node

and has a funcitons:
* expand(): 
    * Moves the position of the blank and returns an array (a list) of Nodes obtained
    * Does not consist parent state done by checking 'Node.a' values 




In [0]:
class Node:
  def __init__(self, state, parent, cost, action):
    self.state = state
    self.parent = parent
    self.g = cost
    self.a = action # Action applied to reach the state L, R, T or D

  # Heuristic: sum of manhattan distances of each tile.. blank included
  def h(self, goal):
    sum = 0
    for i in range(3):
      for j in range(3):
        m, n = np.where(goal == self.state[i][j]) # np.where(a == b) returns the array of positions in the array 'a', where the elements are == b
        m, n = int(m), int(n)
        sum += abs(i - m) + abs(j - n)
    return sum

  def expand(self):
    i, j = np.where(self.state == 0)
    i, j = int(i), int(j) # Current position of the blank -> {0}
    children = []

    # Move the blank one step at a time
    if self.a != 'D' and i - 1 >= 0:
        s = self.state.copy()
        s[i][j] = s[i - 1][j]
        s[i - 1][j] = 0
        children.append(Node(s, self, self.g + 1, 'T'))

    if self.a != 'R' and j - 1 >= 0:
        s = self.state.copy()
        s[i][j] = s[i][j - 1]
        s[i][j - 1] = 0
        children.append(Node(s, self, self.g + 1, 'L'))
    
    if self.a != 'L' and j + 1 <= 2:
        s = self.state.copy()
        s[i][j] = s[i][j + 1]
        s[i][j + 1] = 0
        children.append(Node(s, self, self.g + 1, 'R'))

    if self.a != 'T' and i + 1 <= 2:
        s = self.state.copy()
        s[i][j] = s[i + 1][j]
        s[i + 1][j] = 0
        children.append(Node(s, self, self.g + 1, 'D'))
    return children

in_closed and in_open funcitons checks if the given state is in closed or open list

If is in the open list, updates the value in open list with least f-value node and returns True

In [0]:
def in_closed(s, closed):
  for N in closed:
    if np.allclose(s, N):
      # print(f"found \n{child.state} in closedSet") # Watchhh
      return True
  return False

def in_open(s, openSet, f):
  for N in openSet:
        if np.allclose(s, N[2].state) and f < N[0]:
          openSet.remove(N)
          openSet.append((f, N[1], N[2]))
          hq.heapify(openSet)
          # print("openSet has changed") # Watchhh
          return True
  return False

**A***

Used a *heap* to store the Nodes, the data structure is a priority queue

In [0]:
def A_star(start, goal):
  openSet = []
  closed = []
  unique = count() # Not a very important count..

  f = start.h(goal)
  hq.heappush(openSet, (f, -1, start))
  while openSet:
    f_current, t, current = hq.heappop(openSet) # Node with Least f-value

    if np.allclose(goal, current.state): # np.allclose checks if both the arrays are identical
      print("Solution found")
      return current

    for child in current.expand():
      f_child = child.g + child.h(goal)

      # Check if in closed list or open list
      if not in_closed(child.state, closed) and not in_open(child.state, openSet, f_child):
        # Store Nodes with priority based on the f-value, 'u' is an unique integer to break ties
        hq.heappush(openSet, (f_child, next(unique), child))   

    closed.append(current.state)

  print("Solution not found")

In [0]:
def print_soln(sol):
  if sol.parent == None:
    print(sol.state, '\n')
    return
  print_soln(sol.parent)
  print(sol.state, "\n")

In [19]:
puzzle = np.asarray([[7, 2, 4], [5, 0, 6], [8, 3, 1]])

goal = list(np.arange(1, 9))
goal.append(0)
goal = np.asarray(goal).reshape(3, 3)

start = Node(puzzle, None, 0, '')

print_soln(A_star(start, goal))

Solution found
[[7 2 4]
 [5 0 6]
 [8 3 1]] 

[[7 2 4]
 [5 3 6]
 [8 0 1]] 

[[7 2 4]
 [5 3 6]
 [8 1 0]] 

[[7 2 4]
 [5 3 0]
 [8 1 6]] 

[[7 2 4]
 [5 0 3]
 [8 1 6]] 

[[7 2 4]
 [0 5 3]
 [8 1 6]] 

[[0 2 4]
 [7 5 3]
 [8 1 6]] 

[[2 0 4]
 [7 5 3]
 [8 1 6]] 

[[2 4 0]
 [7 5 3]
 [8 1 6]] 

[[2 4 3]
 [7 5 0]
 [8 1 6]] 

[[2 4 3]
 [7 0 5]
 [8 1 6]] 

[[2 4 3]
 [7 1 5]
 [8 0 6]] 

[[2 4 3]
 [7 1 5]
 [0 8 6]] 

[[2 4 3]
 [0 1 5]
 [7 8 6]] 

[[2 4 3]
 [1 0 5]
 [7 8 6]] 

[[2 0 3]
 [1 4 5]
 [7 8 6]] 

[[0 2 3]
 [1 4 5]
 [7 8 6]] 

[[1 2 3]
 [0 4 5]
 [7 8 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 

